# Table of contents
*Aoife Gregg, June 2023*
1. [Introduction](#introduction)
2. [Importing and processing data](#data)
    1. [Importing data from AWS database](#AWS)
    2. [Processing data](#processing_data)
3. [Email anonymisation](#anonymisation)
4. [Large language models for email analysis](#LLMs)
    1. [Open-source LLM analysis of emails](#open_source_LLM)
    2. [OpenAI LLM analysis of emails](#openai_LLM)
    3. [Comparison of LLM results](#LLM_comparison)

## Introduction <a name="introduction"></a>

This notebook goes through the code used for the Faculty X LocalGlobe project in 2023, including graphs of key results.

The key output of this project was the demonstration of extracting key financial data from investor update emails using a range of large language models. OpenAI gpt-3.5-turbo performed best out of the models tested, but tests were limited due to legal and data security concerns. 

Run this cell to install the relevant packages for this notebook. It assumes a Linux or conda installation (for PyTorch, needed for HuggingFace transformers). I recommend installing and running this code in a virtual environment to avoid version incompatibilities in your system.

In [ ]:
pip install -r requirements.txt

Importing relevant packages.

In [ ]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine,text
import json
import numpy as np
import os
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re
import html
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForCausalLM, pipeline, logging
import spacy
import ast
from sklearn.metrics import accuracy_score,mean_absolute_error
import kaleido

## Importing and processing data<a name="data"></a>

### Importing data from AWS database<a name="AWS"></a>

In [ ]:
alchemyEngine = create_engine(
    os.getenv("DATABASE_URL"),
    pool_recycle=3600
)
db_connection = alchemyEngine.connect()

Run the appropriate cells below to download the data of interest.

In [ ]:
# trello_card = all information contained in Trello cards
sql_query = text("select * from nazare.ingestion.trello_card") 
df_trello = pd.read_sql(sql_query, db_connection)

In [ ]:
# tf_form = typeform survey results
sql_query = text("select * from nazare.ingestion.tf_form")

df_tf = pd.read_sql(sql_query, db_connection)

In [ ]:
#dealroom company data
sql_query = text("select * from nazare.ingestion.dealroom_company")

df_dealroom_company = pd.read_sql(sql_query, db_connection)

In [ ]:
#dealroom funding round data
sql_query_dealroom_fundinground = text("select * from nazare.ingestion.dealroom_fundinground")

df_dealroom_fundinground = pd.read_sql(sql_query_dealroom_fundinground, db_connection)

In [ ]:
# ops_companyfund = data on individual portfolio companies from Google sheet
sql_query = text("select * from nazare.ingestion.ops_companyfund")

df_ops = pd.read_sql(sql_query, db_connection)

### Processing data<a name="processing_data"></a>

#### Email data

Collecting all text comments for all companies from Trello cards, some of which are emails uploaded as comments. Not filtering between emails and other comments as the LLM should be able to cope with the different formats.

In [ ]:
trello_ids = df_trello['short_link_hash'].unique()

# create a dataframe containing the comments on the company along with identifying information

data = []

for trello_id, group in df_trello.groupby(by='short_link_hash'):
    
    # There should only be one company in each group; if not, there has been a problem with labelling in the dataframe    
    assert len(group) == 1, '''
    There is more than one card in the database with this Trello ID, which should not happen. 
    Resolve this issue before continuing.
    '''    
    
    row = group.iloc[0,:]
    
    comments = row['comments']
    
    for comment in comments:
    
        data.append([
            row['name'],
            trello_id,
            comment['date'][:10],
            comment['id'],
            comment['data']['text'],
        ])
        
df_comments = pd.DataFrame(
    data=data,
    columns = ['company_name','trello_id','date','comment_id','text']
)

#### Typeform survey data

Restructuring Typeform survey data to extract results from json to pandas dataframe.

First, removing surveys that were sent out but no-one responded to.

In [ ]:
df_only_answered_surveys = df_tf.loc[df_tf.responses.str.len()>0]

Defining a function to extract survey answers from a json format to a dictionary with the question as keys and the answers as values. 

The json structure varies depending on the question in the survey due to the way it is exported by Typeform.

In [ ]:
def extract_survey_data(fields, response):
    '''Extracts survey answers from json format to a dictionary with the question as keys and answer as values.'''

    questions = {question["ref"]:question["title"] for question in fields}
    
    answers = {}

    for answer in response["answers"]:
        
        question = questions[answer["field"]["ref"]]
        
        if answer["type"] == "choices": #collects multiple choice where multiple answers are allowed (question: concerns)
            if "labels" in answer["choices"]:
                extracted_answer  = answer["choices"]["labels"]
            elif "other" in answer["choices"]:
                extracted_answer = answer["choices"]["other"]
            else:
                raise RunTimeError(f"Answer type {answer['type']} Did not have recognised options: {answer['choices']}. Question was {question}")
                
        else:
            
            match answer["field"]["type"]:
                
                # 'dropdown' collects Meeting Type
                # 'short_text' collects text comments on Excitement, Vision, Market, Product, Team, Timing, Fundraising, Opportunity, Return
                case "dropdown" | "short_text": 
                    extracted_answer = answer["text"]
                    
                # 'opinion' scale collects scales on Excitement, Vision, Market, Product, Team, Timing, Fundraising, Fit, Opportunity    
                # 'rating' collects opinion numbers on another format of Excitement question
                case "opinion_scale" | "rating": 
                    extracted_answer = answer["number"]
                                               
                case "multiple_choice":
                    if answer["choice"]["id"] != "other":
                        extracted_answer  = answer["choice"]["label"] #collects Return and name of person filling out survey
                    else:
                        extracted_answer  = answer["choice"]["other"] # collects name of person filling out survey if they are not in the system                              
                                               
                case _:
                    raise RunTimeError(f"Answer {answer} not recognised. Question was {question}")
        
        # The exact formatting of the questions varies between surveys, so this section uses knowledge of keywords 
        # in the questions to sort all questions into the correct categories with consistent naming.
        
        question_categories = [
            'Excitement',
            'Vision',
            'Opportunity',
            'Team',
            'Fundraising',
            'Fit',
            'Product',
            'Timing',
        ]
        
        for category in question_categories:
            if category.lower() in question.lower():
                if type(extracted_answer) is int:
                    question = f'{category} score'
                elif type(extracted_answer) is str:
                    question = f'{category} text response'
                    
        # these questions have slightly different formatting
        if 'market' in question.lower() and 'timing' not in question.lower():
            if type(extracted_answer) is int:
                question = 'Market score'
            elif type(extracted_answer) is str:
                question = 'Market text response'

        if 'return' in question.lower():
            if 'because' not in question.lower():
                question = 'Return value'
            else:
                question = 'Return text response'

        if 'name' in question.lower() and 'company' not in question.lower():
            question = 'Responder name'

        if 'where are we' in question.lower():
            question = 'Meeting stage'

        if 'concern' in question.lower():
            question = 'Concerns'
                                
        answers[question] = extracted_answer
                                               
    return answers

Using the above function to cycle through all surveys in the AWS database and store the results in a pandas dataframe. 

Then sorting to ensure accuracy when plotting data over time.

In [ ]:
data = [] 
for idx,row in df_only_answered_surveys.iterrows():
    for response in row['responses']:
        
        answer = extract_survey_data(row['fields'], response)
        
        answer.update(row[['company_name','trello_id','created_at','id']].to_dict())
        
        data.append(answer)
        
df_extended_responses = pd.DataFrame(data)

df_extended_responses.sort_values(by=['created_at'],inplace=True) 

#### Dealroom data

In [ ]:
# converting time data to pandas datetime type
df_dealroom_fundinground["day"] = 1 # pandas datetime requires year, month, and day, but dealroom data doesn't include day. Setting the day to the 1st of the month.
df_dealroom_fundinground["time"] = pd.to_datetime(df_dealroom_fundinground[["year","month","day"]])

Choose the company of interest and check that it appears in the dealroom database.

In [ ]:
company_name = 'Example'
mask = df_dealroom_company["name"] == company_name

company_id = df_dealroom_company[mask]["id"].iloc[0]
df_dealroom_company[mask].head() #check that you're looking at the correct company/the company is mentioned in dealroom data

Plot the funding of the company over time, labelled with the funding round type. On a secondary axis, plot the survey scores over time. 

This graph is useful for seeing how opinions over time may be related to the funding a company is receiving.

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

# creating a filter to extract only the data related to the chosen company
company_id_mask = df_dealroom_fundinground["company_id"] == company_id

# plotting the funding over time
fig.add_trace(go.Scatter(x=df_dealroom_fundinground[company_id_mask]['time'],
                         y=df_dealroom_fundinground[company_id_mask]['amount_usd_million'],
                         mode='markers',
                         marker_symbol = 'x-thin',
                         marker_line_width=1,
                         marker_size = 10,
                         text=df_dealroom_fundinground[company_id_mask]['round'],
                         name = 'Funding raised'
                        ),
              secondary_y=False,
              )

# plotting the survey scores over time
numeric_columns = [
    'Excitement score',
    'Fit score',
    'Fundraising score',
    'Market score',
    'Opportunity score',
    'Product score',
    'Team score',
    'Timing score',
    'Vision score',
]

for column in numeric_columns:
    fig.add_trace(go.Scatter(x=df_extended_responses[df_extended_responses['company_name']== company_name]['created_at'].unique(),
                             y=df_extended_responses[df_extended_responses['company_name']== company_name].groupby('created_at')[column].mean(),
                             name = column
                            ),
                  secondary_y=True,
                  )


fig.update_layout(
    title_text=company_name
)


fig.update_xaxes(title_text="Time")
fig.update_yaxes(title_text="Funding raised (million USD)", secondary_y=False)
fig.update_yaxes(title_text="Survey score", secondary_y=True)

fig.show()



The following section demos some ways in which existing internal data can be visualised. 

*Anonymisation: I have removed explanations of exactly what I am plotting here in order to preserve privacy.*

In [ ]:
df_trello['trello_id'] = df_trello['short_link_hash']
df_merged = df_ops.merge(df_tf,how='left',on='trello_id')
df_merged = df_merged.merge(df_trello, how = 'left', on = 'trello_id')

In [ ]:
def replace_blank(value,fill='unknown'):
    if str(value) == '' or str(value) == ' ':
        output = fill
    else:
        output = value
    return output

df_merged['braggy'] = df_merged['braggy'].map(replace_blank)

df_merged.replace('', np.nan, inplace=True)
df_merged.dropna(subset=['company_name','initial_stage','braggy'], inplace=True)

In [ ]:
fig = go.Figure()

x_axis = 'initial_investment'
y_axis = 'current_post_valuation'


for value in df_merged.investment_type.unique():
    df_temp = df_merged[df_merged['investment_type'] == value]
    fig.add_trace(go.Scatter(x=df_temp[x_axis],
                             y=df_temp[y_axis],
                             mode='markers',
                             text=df_temp["company_name"]+', '+ df_temp["initial_stage"],
                             name = value,
                             marker_line_width=1,
                             marker_symbol = 'circle-open',
                            )
                  )

fig.update_layout(
    xaxis_title=x_axis,
    yaxis_title=y_axis,
)


fig2 = go.Figure()
x_axis = 'current_owned'
y_axis = 'current_nav'

for value in df_merged.braggy.unique():
    df_temp = df_merged[df_merged['braggy'] == value]
    fig2.add_trace(go.Scatter(x=df_temp[x_axis],
                             y=df_temp[y_axis],
                             mode='markers',
                             text=df_temp["company_name"]+', '+ df_temp["initial_stage"],
                             name = value,
                             marker_line_width=1,
                             marker_symbol = 'circle-open',
                            )
                  )


fig2.update_layout(
    xaxis_title=x_axis,
    yaxis_title=y_axis,
)


fig3 = go.Figure()
x_axis = 'current_owned'
y_axis = 'current_nav'

fig3.add_trace(go.Scatter(x=df_merged[x_axis],
                         y=df_merged[y_axis],
                         mode='markers',
                         text=df_merged["company_name"]+', '+ df_merged["initial_stage"],
                         marker_line_width=1,
                         marker_symbol = 'circle-open',
                         marker_color = df_merged.latest_excitement_score,
                         marker = dict(
                            colorbar=dict(
                                title="Latest Excitement"
                         )),
                         
                        )
              )

fig3.update_layout(
    xaxis_title=x_axis,
    yaxis_title=y_axis,
)

fig.show()
fig2.show()
fig3.show()
    


## Email anonymisation<a name="anonymisation"></a>

Defining an anonymisation function that takes text and a list of companies and people names. Removes those names, along with a list of LocalGlobe portfolio company names, from the input text. Outputs anonymised text.

In [ ]:
def anonymiser(text, people = None, organisations = None):
    
    '''
    Takes text and optionally a list of people and companies.
    Combines these lists with a list of all portfolio companies in LocalGlobe 
    (extracted from company names in the Typeform survey database).
    Removes any words with length >2 that appears in these lists from the input text
    and replaces it with XXX or similar.
    Returns this anonymised text.
    '''
    
    # use regex to detect  an URL in the form of 'http(s)...' and replace with [URL]
    anonymised_text = re.sub(r'http\S+', '[URL]', text)
    # use regex to detect  an email in the form of '[...]@[...].[...]' and replace with [XXX@XXXX.XX]
    anonymised_text = re.sub(r'[A-Za-z0-9]*@[A-Za-z]*\.?[A-Za-z0-9]*', "XXX@XXXX.XX", anonymised_text)
    
    
    # extracts list of portfolio companies from Typeform survey data
    if organisations is None:
        organisations = df_tf["company_name"].tolist()
    else:
        organisations += df_tf["company_name"].tolist()
    organisations += ['LocalGlobe']
    
    if people is None:
        people = []

    # removes brackets from company and people names to avoid parsing errors
    for count, value in enumerate(organisations):
        organisations[count] = re.sub(r"[\(\)\{\}\[\]]", " ", value)
    for count, value in enumerate(people):
        people[count] = re.sub(r"[\(\)\{\}\[\]]", " ", value)

    # substituting people's names in the text with 'XXX '
    if len(people)>0:
        for person in people:
            for split in person.split(' '):
                # only add names of length >2 to the list as shorter ones remove too much text during the anonymisation process
                if len(split)>2: 
                    split_escaped = re.escape(split) # escaping non-alphanumeric characters
                    # use regex to substitute all names in the input text with 'XXX '
                    anonymised_text = re.sub(rf'(?:^|\W){split_escaped}(?:$|\W|$)', ' XXX ',anonymised_text,flags=re.IGNORECASE)

    # substituting company names in the text with 'XXX '
    if len(organisations)>0:
        for organisation in organisations:
            organisation_escaped = re.escape(organisation)
            anonymised_text = re.sub(rf'(?:^|\W){organisation_escaped}(?:$|\W|$)', ' XXX ',anonymised_text,flags=re.IGNORECASE)


    return anonymised_text


Downloading the English language model used by spaCy for NER. 

In [ ]:
!spacy download en_core_web_sm

Setting up the BERT and spaCy NER models. Both BERT and spaCy methods are used as BERT generally performs better, but sometimes misses out people's names. Adding in an additional spaCy process (which is generally quick) helps to pick up those missed names.

In [ ]:
# Using HuggingFace transformers to access BERT model
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large-finetuned-conll03-english")
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-large-finetuned-conll03-english")
nlp = pipeline("ner", model = model, tokenizer=tokenizer,aggregation_strategy = "simple")

nlp_spacy = spacy.load("en_core_web_sm")

The following cells use text from **Trello comments** for a given company and use BERT and spaCy for named entity recognition to read through text and identify company and people names.

In [ ]:
company_name = 'example' # set the company of interest 
company_trello_id = df_tf[df_tf['company_name'].str.contains(company_name)]['trello_id'].iloc[0] # find the company Trello ID from its name

df_comments_company = df_comments[df_comments['trello_id']==company_trello_id]

orgs = []
people = []


# for each comment in the dataframe, extract the text and run BERT NER to identify company and people names, then append them to a list.
# we are processing each comment individually as this improves NER performance
for comment_text in df_comments_company['text']:
    ner_results = nlp(comment_text) # ner_results contains list of all 'named entities' identified by BERT NER
    for result in ner_results:
        if result['entity_group'] == 'PER': # if the named entity is a person
            if result['word'] not in people:
                people.append(result['word'])
        if result['entity_group'] == 'ORG': # if the named entity is a company
            if result['word'] not in orgs:
                orgs.append(result['word'])

    
# Read all comments into one text variable for spaCy analysis and then anonymisation.
text_data = "\n".join(df_comments_company['text'])


# Repeat the NER process with spaCy NER to pick up more people's names and add them to the list of people
doc = nlp_spacy(text_data)

types = ["PERSON"]
for token in doc:
    if token.ent_type_ in types and token.text not in people:
        people.append(token.text)

# Input the collected people's and company names into the anonymisation function and print the anonymised text.
anonymised_text = anonymiser(text_data,people=people,organisations=orgs)
print(anonymised_text)



## Large language models for email analysis<a name="LLMs"></a>

### Open-source LLM analysis of emails<a name="open_source_LLM"></a>

The open-source LLM models are tested using HuggingFace transformers. First, choose the model to use by running one of the below cells or adding in another HF model (https://huggingface.co/models).

If you're using a **quantised** model (here, these are the models which use AutoGPTQ), first install and import the required packages. These packages require CUDA, so check/install this beforehand. Again, I recommend that you run this code in a virtual environment to avoid introducing version incompatibilities to your system. There is no need to install/import these if you are not using a quantized model.

In [ ]:
!pip install -r requirements_quantised.txt

In [ ]:
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

A quantised version of **Falcon 7B** is small enough to run on a local computer, but does not produce consistent results.

In [ ]:
MODEL = "TheBloke/falcon-7b-instruct-GPTQ"

model_basename = "gptq_model-4bit-64g"

tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
        MODEL,
        model_basename=model_basename,
        device="cuda:0",                                        
        use_safetensors=True,
        use_triton=False,
        trust_remote_code=True,
        quantize_config=None
)

**RedPajama** requires roughly >20 GB VRAM to run.

In [ ]:
MODEL = "togethercomputer/RedPajama-INCITE-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

model = AutoModelForCausalLM.from_quantized(
    MODEL, 
    device="cuda:0", 
    use_safetensors=True, 
    use_triton=False, 
    trust_remote_code=True
)

**Vicuna 33B** provides better results than most other open-source models tested, but takes a lot of memory to run. Here, a quantised version (i.e. modified to run using less memory) is used but this still requires a GPU with >25GB VRAM.

Typical errors: Vicuna 33B cannot reliably distinguish between similar metrics e.g. monthly total revenue vs monthly subscription revenue.

In [ ]:
MODEL = "TheBloke/vicuna-33B-GPTQ"
model_basename = "vicuna-33b-GPTQ-4bit--1g.act.order"

model = AutoGPTQForCausalLM.from_quantized(
    MODEL,
    model_basename=model_basename,
    device="cuda:0",
    use_safetensors=True,
    use_triton=False,
    trust_remote_code=True,
    quantize_config=None
)

**Falcon 40B** provides some of the best results of the open-source models tested, but takes a lot of memory to run. Here, a quantised version (i.e. modified to run using less memory) is used but this still requires a GPU with >40GB VRAM.

Typical errors: Falcon 40B is prone to hallucination, so do not trust it on documents where the answer may not be present.

*Note (August 2023): Since working on this project, Meta has released a new open-source LLM (Llama 2, which has 70B parameters and Meta claims outperforms GPT-3.5) that may perform better than Falcon 40B.*

In [ ]:
MODEL = "TheBloke/falcon-40b-instruct-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    MODEL, 
    device="cuda:0", 
    use_safetensors=True, 
    use_triton=False, 
    trust_remote_code=True
)

Next, define a function to extract information from body of text using the selected model.

In [ ]:
def extract_information(model_choice,tokenizer_choice,question, document,end_token_length = 0):
    '''
    Run a pre-defined model and tokenizer on input document text.
    If end_token_length is provided, it will cut off that many characters from the end of the 
    response given by the model.
    '''
    
    prompt = f"""User: {question} \n Context: {document} \n \n Assistant:\n """

    input_ids = tokenizer_choice(prompt, return_tensors='pt').input_ids.cuda()
    output = model_choice.generate(inputs=input_ids, temperature=0, max_new_tokens=512)
    full_output_text = tokenizer.decode(output[0])
    input_length = len(tokenizer.decode(input_ids[0]))
    results_output_text = full_output_text[input_length:-end_token_length] 
    torch.cuda.empty_cache()
    return results_output_text

Choose which company's comments you're looking at.

First, check the Trello ID of the company you're looking for.

In [ ]:
company_name = 'example' # set the company of interest 
name_mask = df_comments["company_name"].str.contains(company_name)
df_comments[name_mask]

Due to multiple Trello boards containing the same company names, check which board is actually referring to the company you want. If the correct company is at the top of the results above, use the automatic company ID below. If not, either manually set it or set the *iloc* index to the correct value.

In [ ]:
company_trello_id = df_comments[name_mask]['trello_id'].iloc[0] # set the company Trello ID from its name

Define the question to run on these comments and run the selected model on each comment one at a time, storing these results as text in a json file.

In [ ]:
question = """You are an intelligent assistant answering questions based on information from documents.
Answer the following questions using the context given.
 1. What is the total monthly recurring revenue? \n
 2. What is the runway in months? \n
 3. What is the total working capital? \n
 4. What is the total number of full time employees? \n
 5. What is the monthly burn rate? \n
 6. On what date was this document written? \n
 7. What is the total annual recurring revenue? 
 """    

results = [
    {
        'model': model,
        'question' : question,    
    }
]

for idx, row in df_comments[df_comments['trello_id']==company_trello_id].iterrows():
    answers = extract_information(model,tokenizer,question,row['text'])
    
    result = {
        'answers' : answers,
    }
    
    # storing the metadata along with the answers
    # note the date here refers to the date from the Trello comment metadata, not the date the email was written
    result.update(row[['date','trello_id','comment_id']].to_dict())
    
    results.append(result)

with open(f"{company_name}.json", "w") as file:
    json.dump(results, file)

The text answers then need to be processed in order to plot them. The formatting varies significantly for different models used, but we can expect some consistency if the questions are asked in the form of a numbered list: most models will then also output a numbered list, which can be processed by detecting the numbers and associating them with the correct question. 

I have not yet seen a model that will correctly answer the questions but incorrectly number the answers, so this processing method should not lose any useful information.

In [ ]:
with open(f'{company_name}.json','r') as file:
    data = json.load(file)

data = data[1:]  # remove the element containing information about the run

extracted_data = []
params = ['monthly_revenue', 'runway', 'capital', 'FTEs', 'burn_rate', 'date','annual_revenue'] 

# iterating over the json containing the LLM results for the set of emails for a given company
for element in data:
    text = element['answers']
    extracted_strings = {}

    #I know the format of my text is a string in the form of a numbered list from 1. to 7. 
    #This section splits the string into each bullet point.
    text = re.sub(',','',text) #removing commas to interpret numbers more easily: £100,000 -> £100000
    split_text = re.split(r'(\s|^)\d\.\s',text) #splits the text up by bullet points (1. , 2. , 3. , etc)
    split_text = [x for x in split_text if x] #removes any empty splits
    split_text = [x for x in split_text if x.isspace() is False] # removes any splits which are just whitespace.

    # Check that I have extracted the correct number of answers.
    # I know that these results should be answering the 7 questions defined in 'params' in the correct order.
    # If the length doesn't match, the LLM question-answering has failed.
    if len(split_text) == len(params):       
        # assign the answers to the questions
        for count, param in enumerate(params):
            extracted_strings[param] = split_text[count]

        # The bullet points contain text as well as numbers. This section extracts the first number given in each bullet point.
        # Note that this action is not needed for dates, so it is skipped there.
        extracted_numerical_values = {}
        for key, value in extracted_strings.items():
            if key != "date":
                numerical_value = re.findall(r'(\d+(?:\.\d+)?)', value) 
                #contains_million checks if the string contains M, m, or million (m or M only as individual letters)
                contains_million = re.findall(r"(\d+(?:\.\d+)?)\s*(?:m\b|million)",value, re.IGNORECASE)
                if len(numerical_value) > 0:
                    if len(contains_million)>0:
                        extracted_numerical_values[key] = float(numerical_value[0])*1e6
                    else:
                        extracted_numerical_values[key] = float(numerical_value[0])
                else:
                    extracted_numerical_values[key] = np.nan
            else:
                extracted_numerical_values[key] = value

        extracted_data.append(extracted_numerical_values)

    else:
        for count, param in enumerate(params):
            extracted_strings[param] = np.nan
        extracted_data.append(extracted_strings)
        
        
df_extracted_data = pd.DataFrame(extracted_data)

Importing and cleaning reference data on the correct answers (to check accuracy of LLM). These data were collected by manually reading a small set of emails.

In [ ]:
comparison_data = pd.read_csv("example.csv")

# some columns have different names in the comparison_data set. This dictionary is used to rename them.
column_names = {
    'monthly recurring revenue' : 'MRR',
    'burn rate' : 'burn_rate',
    'annual recurring revenue' : 'ARR',
}

df_comparison_data = comparison_data[['monthly recurring revenue','runway','capital','FTEs','burn rate','annual recurring revenue']].replace(',','',regex=True)

contains_million_mask = df_comparison_data.apply(lambda x: x.str.contains('m', case=False))

df_comparison_data= df_comparison_data.replace('[^.0-9]', '', regex=True).astype(float,errors = 'ignore')

df_comparison_data.replace('',np.nan,inplace= True)
df_comparison_data = df_comparison_data.astype('float')

df_comparison_data = df_comparison_data.mask(contains_million_mask,df_comparison_data*1e6)
df_comparison_data.rename(columns=column_names, inplace=True)


Calculate the accuracy of the LLM data extraction by comparing to the manually-extracted results.

In [ ]:
common_columns = list(set(df_extracted_data.columns).intersection(df_comparison_data.columns))

avg_acc = 0.0
for c in common_columns:
    c_acc = accuracy_score(df_extracted_data[c].round(), df_comparison_data[c].round())
    print(f'{c} accuracy: {c_acc:.2f}')
    avg_acc += c_acc/len(common_columns)
    
print(f'Average accuracy: {avg_acc:.2f}')

### OpenAI LLM analysis of emails<a name="openai_LLM"></a>

Using gpt-3.5-turbo model on Azure OpenAI service.

In [ ]:
import openai

openai.api_type = "azure"
openai.api_base = "https://faculty-openai.openai.azure.com/"
openai.api_version = "2023-05-15"
openai.api_key = os.environ["OPENAI_API_KEY"]


results = {}

# For each comment for the specified company, input the comment text as content, ask questions about it, and store the answer.
for idx, row in df_comments[df_comments['trello_id']==company_trello_id].iterrows():
    completion = openai.ChatCompletion.create(
        engine=os.environ["OPENAI_ENGINE"],
        messages=[
            {"role": "system", "content": row['text']},
            {"role": "user", "content": """What are the current 
             total monthly revenue (which can also be called MRR), 
             monthly net cash burn, 
             number of employees employees, 
             cash in bank, 
             total annual revenue (which can also be called ARR),
             runway,
             and date on which this email was sent? 
             Output this information in the format of a python dictionary.
             Do not include any additional comments.
             If the information is not present, return "unknown".
             """}
        ],
        temperature = 0,
    )
    results[row['comment_id']] = completion.choices[0].message['content']
    print(completion.choices[0].message['content'])

# save the results
with open(f'OpenAI_{company_name}.json', 'w') as f:
    json.dump(results, f)

Importing results saved from running OpenAI gpt-3.5-turbo above and processing:
- Using *ast* package to read output strings as a dictionary.
- If data on cash is written with 'million' or 'M' or 'm', extract the number only and multiply it by 1,000,000.
- Generally strip excess text and punctuation from results.

*Note that the ingestion from OpenAI gpt-3.5-turbo is a lot easier as it can be asked to output the answer in the form of a dictionary without affecting the quality of its answers.*

In [ ]:
with open(f'OpenAI_{company_name}.json', 'r') as f:
    data = json.load(f)

extracted_results = []

reference_keys = ['MRR','burn_rate','FTEs','capital','ARR','runway','date']
numerical_columns =['MRR','burn_rate','FTEs','capital','ARR','runway'] 

#iterating over the extracted data from all comments for a specified company
for key in data:
    # converting the result from a string to a dictionary
    result = ast.literal_eval(data[key])
    # OpenAI is not always consistent with its key naming when generating dictionaries.
    # Here, we replace the OpenAI keys with the correct keys.
    result_correct_keys = {}
    for count, open_ai_key in enumerate(result.keys()):
        result_correct_keys[reference_keys[count]] = result[open_ai_key]
    extracted_results.append(result_correct_keys)

# cleaning the extracted data to convert the strings to numeric data
df = pd.DataFrame.from_dict(extracted_results).replace(',','',regex=True)
contains_million_mask = df[numerical_columns].apply(lambda x: x.str.contains(r"(\d+(?:\.\d+)?)\s*(?:m\b|million)", case=False))
contains_million_mask.fillna(False,inplace=True)

df[numerical_columns]= df[numerical_columns].replace('[^.0-9]', '', regex=True)
df.replace('',np.nan,inplace= True)
df[numerical_columns] = df[numerical_columns].astype('float')


df[numerical_columns] = df[numerical_columns].mask(contains_million_mask,df[numerical_columns]*1e6)

Importing reference data on the correct answers (to check accuracy of LLM). These data were collected by manually reading a small set of emails.

In [ ]:
comparison_data = pd.read_csv("example.csv")
column_names = {
    'monthly recurring revenue' : 'MRR',
    'burn rate' : 'burn_rate',
    'annual recurring revenue' : 'ARR',
}

df_comparison_data = comparison_data[['monthly recurring revenue','runway','capital','FTEs','burn rate','annual recurring revenue']].replace(',','',regex=True)

contains_million_mask = df_comparison_data.apply(lambda x: x.str.contains('m', case=False))

df_comparison_data= df_comparison_data.replace('[^.0-9]', '', regex=True).astype(float,errors = 'ignore')

df_comparison_data.replace('',np.nan,inplace= True)
df_comparison_data = df_comparison_data.astype('float')

df_comparison_data = df_comparison_data.mask(contains_million_mask,df_comparison_data*1e6)
df_comparison_data.rename(columns=column_names, inplace=True)

Plotting the **OpenAI** LLM-extracted results along with the real data.

In [ ]:
pd.options.plotting.backend = "plotly"


fig = make_subplots(rows=3, cols=2, subplot_titles=(
    'Monthly revenue ($)',  
    'Full-time employees',
    'Runway (months)',
    'Burn rate ($)',
    'Capital ($)',
    'ARR ($)',
    ),
    horizontal_spacing = 0.07,
    vertical_spacing = 0.06,
)

plot_params = ['MRR', 'runway', 'capital', 'FTEs', 'burn_rate','ARR'] 

for count, param in enumerate(plot_params):
    if count < 3:
        fig.add_trace(go.Scatter(
            x=df.index,
            y=df[param],
            name = 'Extracted by OpenAI',
            mode = 'markers',
            marker_color='rgba(9, 144, 185, 1)',
            legendgroup="LLM",
            showlegend = count == 0,
        ),
              row=count+1, col=1)
        fig.add_trace(go.Scatter(
            x=df_comparison_data.index,
            y=df_comparison_data[param],
            name = 'Manually labelled', 
            mode = 'markers',
            marker_line_color='rgba(224, 13, 45, 1)',
            marker_symbol = 'x-thin',
            marker_line_width=1,
            marker_size = 10,
            legendgroup="actual",
            showlegend = count == 0,
        ),
              row=count+1, col=1)
    else:
        fig.add_trace(go.Scatter(
            x=df.index, 
            y=df[param],
            name = 'OpenAI', 
            mode = 'markers',
            marker_color='rgba(9, 144, 185, 1)',
            legendgroup="LLM",
            showlegend = count == 0,
        ),
              row=count-2, col=2)
        fig.add_trace(go.Scatter(
            x=df_comparison_data.index, 
            y=df_comparison_data[param],
            name = 'Reference', 
            mode = 'markers',
            marker_line_color='rgba(224, 13, 45, 1)',
            marker_symbol = 'x-thin',
            marker_line_width=1,
            marker_size = 10,
            legendgroup="actual",
            showlegend = count == 0,
        ),
              row=count-2, col=2)
        

fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.05,
        xanchor="left",
        x=0.21,
        font=dict(size= 20)
    ),
    width=1000,
    height=800,
    font=dict(
        size = 18
    ),
                 
)

fig.update_xaxes(visible=False)
fig.show()

Calculating the accuracy of OpenAI data extraction by comparing its results to manually-extracted data.

In [ ]:
common_columns = list(set(df.columns).intersection(df_comparison_data.columns))

for c in common_columns:
    c_acc = accuracy_score(df[c].round(), df_comparison_data[c].round())
    print(f'{c} accuracy: {c_acc:.2f}')

### Comparison of LLM results<a name="LLM_comparison"></a>

Summary of accuracy from a range of models (example data copied from accuracies generated using above code):

In [ ]:
models = ['OpenAI','Falcon 40B','Vicuna 33B','RedPajama']

results = {
    'OpenAI' : {
        'annual<br>revenue' : 1,
        'burn<br>rate' : 1,
        'capital' : 1,
        'employees' : 1,
        'monthly<br>revenue' : 1,
        'runway' : 1,
    },
    'Falcon 40B' : {
        'annual<br>revenue' : 0.78,
        'burn<br>rate' : 1,
        'capital' : 1,
        'employees' : 0.39,
        'monthly<br>revenue' : 0.91,
        'runway' : 1,
    },
    'Vicuna 33B' : {
        'annual<br>revenue' : 0.78,
        'burn<br>rate' : 1,
        'capital' : 1,
        'employees' : 0.91,
        'monthly<br>revenue' : 0.17,
        'runway' : 1,
    },
    'RedPajama' : {
        'annual<br>revenue' : 0.22,
        'burn<br>rate' : 0.7,
        'capital' : 0.7,
        'employees' : 0.09,
        'monthly<br>revenue' : 0.61,
        'runway' : 0.78,
    },
}

df_summary = pd.DataFrame.from_dict(results)

fig = go.Figure()



fig.add_trace(go.Scatter(x=df_summary.index, y=df_summary['Falcon 40B'],
                    mode='markers',
                    marker_size = 15,
                    marker_line_color='rgba(9, 144, 185, 1)',
                    marker_line_width=1.5,
                    marker_symbol = 'x-thin',
                    name = 'Falcon 40B',
                    legendgrouptitle_text="Open source",
                    legendgroup = 'Open source'
                        ))

fig.add_trace(go.Scatter(x=df_summary.index, y=df_summary['Vicuna 33B'],
                    mode='markers',
                    marker_size = 15,
                    marker_line_color='rgba(9, 144, 185, 1)',
                    marker_line_width=1.5,
                    marker_symbol = 'cross-thin',
                    name = 'Vicuna 33B',
                    legendgrouptitle_text="Open source",
                    legendgroup = 'Open source'
                        ))

fig.add_trace(go.Scatter(x=df_summary.index, y=df_summary['RedPajama'],
                    mode='markers',
                    marker_size = 10,
                    marker_color='rgba(9, 144, 185, 1)',
                    marker_line_width=1.5,
                    marker_symbol = 'circle-open',
                    name = 'RedPajama<br>INCITE-7B',
                    legendgrouptitle_text="Open source",
                    legendgroup = 'Open source'
                        ))

fig.add_trace(go.Scatter(x=df_summary.index, y=df_summary['OpenAI'],
                    mode='markers',
                    marker_size = 10,
                    marker_color = 'rgba(235, 118, 12, 1)',
                    name = 'OpenAI<br>gpt-3.5-turbo',
                    legendgrouptitle_text="Proprietary",
                    legendgroup = 'Proprietary'
                        ))


fig.update_layout(
    autosize=False,
    yaxis_title="Accuracy",
    xaxis_title = 'Extracted metrics',
#     legend_title="Model",
    font = dict(
        size = 14
    ),
    legend_traceorder='grouped+reversed'

)


fig.show()
fig.write_image("LLM_results_summary.svg")